<a href="https://colab.research.google.com/github/EasonChen78/machine_learning/blob/main/20230505_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

84125825/84125825 [==============================] - 10s 0us/step


In [2]:
dataset

'/root/.keras/datasets/aclImdb.tar.gz'

In [3]:
import glob
glob.glob("/root/.keras/datasets/aclImdb/train/*")

['/root/.keras/datasets/aclImdb/train/urls_neg.txt',
 '/root/.keras/datasets/aclImdb/train/urls_unsup.txt',
 '/root/.keras/datasets/aclImdb/train/unsupBow.feat',
 '/root/.keras/datasets/aclImdb/train/urls_pos.txt',
 '/root/.keras/datasets/aclImdb/train/unsup',
 '/root/.keras/datasets/aclImdb/train/neg',
 '/root/.keras/datasets/aclImdb/train/labeledBow.feat',
 '/root/.keras/datasets/aclImdb/train/pos']

In [4]:
import os
os.path.dirname(dataset)

'/root/.keras/datasets'

In [5]:
import os
import glob
import pandas as pd
def getdata(mid):
    dn = os.path.dirname(dataset)
    posfn = glob.glob(os.path.join(dn, "aclImdb", mid, "pos", "*"))
    negfn = glob.glob(os.path.join(dn, "aclImdb", mid, "neg", "*"))
    contents = []
    for fn in posfn + negfn:
        with open(fn, encoding="utf-8") as f:
            contents.append(f.read())
    df = pd.DataFrame({
        "content":contents,
        "sentiment":[1] * len(posfn) + [0] * len(negfn)
    })
    return df
train_df = getdata("train")
test_df = getdata("test")

In [6]:
test_df

,content,sentiment
0,"Great just great! The West Coast got ""Dirty"" H...",1
1,This is another case of Hollywood Arrogance pr...,1
2,I was never all that impressed by Night Galler...,1
3,"Originally filmed in 1999 as a TV pilot, ""Mulh...",1
4,I was watching an NFL game and started surfing...,1
...,...,...
24995,A wildly uneven film where the major problem i...,0
24996,"This movie is not worth seeing, at least not a...",0
24997,"This starts out interestingly, as there's a ca...",0
24998,The first season was great - good mix of the j...,0


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalMaxPooling1D, Dense
layers = [
    Embedding(input_dim=3001, output_dim=100, mask_zero=True, input_length=512),
    GlobalMaxPooling1D(),
    Dense(2, activation="softmax")
]

model = Sequential(layers)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 512, 100)          300100    
                                                                 
 global_max_pooling1d (Globa  (None, 100)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 2)                 202       
                                                                 
Total params: 300,302
Trainable params: 300,302
Non-trainable params: 0
_________________________________________________________________


In [9]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(loss=SparseCategoricalCrossentropy(),
       metrics=["accuracy"],
       optimizer="adam")

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000)
tok.fit_on_texts(train_df["content"])

In [ ]:
tok.word_index

In [13]:
import pandas as pd 
x_train_sqe = tok.texts_to_sequences(train_df["content"])
x_test_sqe = tok.texts_to_sequences(test_df["content"])
pd.DataFrame(x_train_sqe)

,0,1,2,3,4,5,6,7,8,9,...,1806,1807,1808,1809,1810,1811,1812,1813,1814,1815
0,2537,6,3,52,951,19,523,31,745.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,83,2,10,59,37,5,132,12,143.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,83,4,29,10,241,3,726,425,911.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,44,1,99,23,5,27,2410,1614,2735.0,23.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,17,16,1,422,1640,1160,339,15.0,46.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,10,781,86,11,17,45,424,54,1515.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24996,1,230,119,3,173,4,422,1593,15.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24997,1,88,1120,151,10,67,132,15,11.0,750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24998,11,695,19,6,368,370,2,42,251.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train_sqe, maxlen=512)
x_test_pad = pad_sequences(x_test_sqe, maxlen=512)
pd.DataFrame(x_train_pad)

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0,0,0,0,0,0,0,0,0,0,...,15,745,14,70,14,1211,17,448,690,155
1,0,0,0,0,0,0,0,0,0,0,...,187,9,6,2,10,437,22,101,35,96
2,0,0,0,0,0,0,0,0,0,0,...,4,1,42,144,110,1321,344,42,144,110
3,0,0,0,0,0,0,0,0,0,0,...,145,34,25,157,734,8,1,1614,1210,179
4,0,0,0,0,0,0,0,0,0,0,...,106,2,13,3,49,2997,15,8,11,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,...,5,116,11,18,213,20,222,161,8,47
24996,0,0,0,0,0,0,0,0,0,0,...,1184,12,141,66,422,154,221,25,217,598
24997,0,0,0,0,0,0,0,0,0,0,...,7,2991,1844,216,278,2,9,158,1171,98
24998,0,0,0,0,0,0,0,0,0,0,...,193,85,42,2331,2,750,89,435,126,55


In [16]:
import numpy as np
y_train = np.array(train_df["sentiment"])
y_test = np.array(test_df["sentiment"])

In [17]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
c = [
    ModelCheckpoint("nlp.h5", save_best_only=True),
    EarlyStopping(patience=5, restore_best_weights=True)
]
model.fit(x_train_pad,
     y_train,
     batch_size=100,
     epochs=40,
     validation_split=0.1,
     callbacks=c)

Epoch 1/40
225/225 [==============================] - 36s 136ms/step - loss: 0.5786 - accuracy: 0.7203 - val_loss: 0.4899 - val_accuracy: 0.7716
Epoch 2/40
225/225 [==============================] - 12s 54ms/step - loss: 0.3432 - accuracy: 0.8660 - val_loss: 0.3834 - val_accuracy: 0.8232
Epoch 3/40
225/225 [==============================] - 5s 22ms/step - loss: 0.2786 - accuracy: 0.8881 - val_loss: 0.3471 - val_accuracy: 0.8452
Epoch 4/40
225/225 [==============================] - 6s 27ms/step - loss: 0.2442 - accuracy: 0.9031 - val_loss: 0.3666 - val_accuracy: 0.8340
Epoch 5/40
225/225 [==============================] - 3s 14ms/step - loss: 0.2185 - accuracy: 0.9164 - val_loss: 0.3456 - val_accuracy: 0.8476
Epoch 6/40
225/225 [==============================] - 2s 9ms/step - loss: 0.1976 - accuracy: 0.9259 - val_loss: 0.3491 - val_accuracy: 0.8464
Epoch 7/40
225/225 [==============================] - 3s 13ms/step - loss: 0.1791 - accuracy: 0.9356 - val_loss: 0.3616 - val_accuracy: 0.84

In [20]:
model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 2s 2ms/step - loss: 0.2929 - accuracy: 0.8760


[0.2929280698299408, 0.8759999871253967]

In [21]:
review = "I love this movie" #@param {type:"string"}
sqe = tok.texts_to_sequences([review])
pad = pad_sequences(sqe, maxlen=512)
pre = model.predict(pad)
prob = pre[0]
trans = ["neg", "pos"]
for p, label in zip(prob, trans):
    print(label, "的機率是:", p)

1/1 [==============================] - 0s 73ms/step
neg 的機率是: 0.2730782
pos 的機率是: 0.7269218
